In [68]:
# super comms script
import serial
from time import sleep
import math

from tqdm import *
import json

In [70]:
def set_target(motor, location, ser, output=True):
    if ser.is_open:
        if motor =='A':
            ser.write(b'A')
        else:
            ser.write(b'B')
        
        target_bytes = location.to_bytes(4, byteorder='big')
        #print(target_bytes)
        ser.write(target_bytes)
        sleep(0.02)
        while(ser.in_waiting > 0):
            b = ser.read()
            if output:
                print(b.decode('ascii'), end='')
    else:
        raise Exception("Serial is not open!")

In [71]:
def get_debug(ser):
    if ser.is_open:
        ser.write(b'D')
        sleep(0.02)
        while(ser.in_waiting > 0):
            b = ser.read()
            print(b.decode('ascii'), end='')
    
        print("---")
    else:
        raise Exception("Serial is not open!")

In [72]:
def gogogo(ser, wait=False, output=True):
    if ser.is_open:
        ser.write(b'G')
        sleep(0.02)
        
        if output:
            print("--- Making a move ---")
        
        if wait:
            end_found = False
            while not end_found:
                sleep(0.002)

                while(ser.in_waiting > 0):
                    b = ser.readline().decode('ascii')
                    if output:
                        print(b)
                    if "move-end" in b:
                        end_found = True
                
        else:
            while(ser.in_waiting > 0):
                b = ser.read()
                print(b.decode('ascii'), end='')
    
            
    else:
        raise Exception("Serial is not open!")

In [73]:
def stop(ser):
    if ser.is_open:
        ser.write(b'S')
        sleep(0.1)
        while(ser.in_waiting > 0):
            b = ser.read()
            print(b.decode('ascii'), end='')
    
        print("---")
    else:
        raise Exception("Serial is not open!")

In [74]:
def penup(ser):
    if ser.is_open:
        ser.write(b'C')
        sleep(0.1)
        while(ser.in_waiting > 0):
            b = ser.read()
            #print(b.decode('ascii'), end='')
    
        #print("---")
    else:
        raise Exception("Serial is not open!")

def pendown(ser):
    if ser.is_open:
        ser.write(b'X')
        sleep(0.1)
        while(ser.in_waiting > 0):
            b = ser.read()
            #print(b.decode('ascii'), end='')
    
        #print("---")
    else:
        raise Exception("Serial is not open!")

In [75]:
def reset(ser, output=True):
    if ser.is_open:
        ser.write(b'R')
        sleep(0.5)
        while(ser.in_waiting > 0):
            b = ser.read()
            if output:
                print(b.decode('ascii'), end='')

    else:
        raise Exception("Serial is not open!")

In [166]:
ser = serial.Serial('/dev/cu.usbserial-141240', baudrate=115200)  # open serial port
print(ser.name)         # check which port was really used

/dev/cu.usbserial-141240


In [172]:
get_debug(ser)

!-- I received: D
!--- Debug ---
!--- A length = 0
!--- B length = 0
!--- A target length = 5000
!--- B target length = 0
---


In [208]:

# Start with thing at home position!
#reset(ser)

target_coord = (300,200)
reset_point = (800,800)

target_lengths = translate_xy_to_ab(target_coord)
travel_lengths = (reset_point[0] - target_lengths[0], reset_point[1] - target_lengths[1])

a_step_mm = 10000/125
b_step_mm = 10000/125

travel_steps = (int(travel_lengths[0] * a_step_mm), int(travel_lengths[1] * b_step_mm))

set_target("A", travel_steps[0], ser, output=True)
set_target("B", travel_steps[1], ser, output=True)
gogogo(ser, wait=True)

!-- I received: A
!--- A target length = 35155
!-- I received: B
!--- B target length = 8062
--- Making a move ---
!-- I received: G

!--- Running ---

!-- Reached move-end ---



In [209]:
set_target("A", 0, ser, output=True)
set_target("B", 0, ser, output=True)

#set_target("A", 132, ser, output=True)
#set_target("B", 9121, ser, output=True)
gogogo(ser, wait=True)



!-- I received: A
!--- A target length = 0
!-- I received: B
!--- B target length = 0
--- Making a move ---
!-- I received: G

!--- Running ---

!-- Reached move-end ---



In [183]:
gogogo(ser, wait=True)

--- Making a move ---
!--- B target length = 5000

!-- I received: G

!--- Running ---

!-- Reached move-end ---



In [ ]:
abpath = [
    (5000,5000),
    (10000,10000),
    (0, 10000),
    (1000, 0)
]
counter = 0
for coord in abpath:
    counter += 1
    print("Step %s of %s (%s)" % (counter, len(abpath), 100*counter/len(abpath)))
    set_target('A', coord[0], ser, output=False)
    set_target('B', coord[1], ser, output=False)
    gogogo(ser, wait=True, output=False)

In [ ]:
ser.close()

In [82]:
reset(ser)

!--- B target length = 17723
!-- I received: G
!--- Running ---
!-- Reached move-end ---
!-- I received: R


In [197]:
def translate_xy_to_ab(coord):
    x = coord[0]
    y = coord[1]
    a_len = math.sqrt(x**2 + y**2)
    b_len = math.sqrt((MAX_WIDTH-x)**2 + y**2)
    
    return [a_len, b_len]

def translate_ab_to_xy(lengths):
    a = lengths[0]
    b = lengths[1]
    
    # Cosine rule!
    #cos(left) =  (a**2 + MAX_WIDTH**2 - b**2) / (2 * a * MAX_WIDTH)
    
    try:
        left_angle = math.acos((a**2 + MAX_WIDTH**2 - b**2) / (2 * a * MAX_WIDTH))
    except Exception as e:
        # This specifically happens if the values just arn't a triangle!
        # i.e. consider maxwidth = 100, left length = 10, right = 10... one of
        # the wires must have broken!
        print("Not a triangle!")
        print((a**2 + MAX_WIDTH**2 - b**2) / (2 * a * MAX_WIDTH))
        raise e
        
    #print(left_angle) # in radians, remember.
    
    # sin(left) = opp / hyp
    # cos(right) = adj / hyp
    # hyp is 'a'
    # Lack of precision here - chop to mm. Rounding 'down'
    y = int(math.sin(left_angle) * a) 
    x = int(math.cos(left_angle) * a)
    
    return [x,y]

In [85]:
# Math time
MAX_WIDTH = 970

a_scale = 10000/130
b_scale = 10000/125

# 0,0 is furthest, then up is less (?)

real_start_mm = (800,800)

orig_length = (real_start_mm[0] * a_scale, real_start_mm[1] * b_scale)
print(orig_length)
xy_path = [
    (500, 390),
    #(500,500),
    #(600,400),
    'HOME'
]

ab_path = []

for coord in xy_path:
    if coord=='HOME':
        movement = (0,0)
    else:
        short_ab_mm = translate_xy_to_ab(coord)
        #print(short_ab_mm)
        short_ab_steps = (short_ab_mm[0] * a_scale, short_ab_mm[1] * b_scale)
        #print(short_ab_steps)
        movement = (int(orig_length[0] - short_ab_steps[0]),int( orig_length[1] - short_ab_steps[1]))
    print("Going -> %s" % (movement,))
    ab_path.append(movement)

(61538.46153846154, 64000.0)
Going -> (12760, 15141)
Going -> (0, 0)


In [159]:
ser = serial.Serial('/dev/cu.usbserial-141210', baudrate=115200)  # open serial port
print(ser.name)   


/dev/cu.usbserial-141210


In [160]:
reset(ser)

!-- I received: R


In [161]:
get_debug(ser)

!-- I received: D
!--- Debug ---
!--- A length = 0
!--- B length = 0
!--- A target length = 0
!--- B target length = 0
---


In [86]:
counter = 0
for coord in ab_path:
    counter += 1
    print("Step %s of %s (%s)" % (counter, len(ab_path), 100*counter/len(ab_path)))
    set_target('A', coord[0], ser, output=False)
    set_target('B', coord[1], ser, output=False)
    gogogo(ser, wait=True, output=False)

Step 1 of 2 (50.0)
Step 2 of 2 (100.0)


In [62]:
ser.close()

In [214]:
with open("spiro.json") as fp:
    paths = json.load(fp)

MAX_WIDTH = 970
offset_x = 300
offset_y = 50

scale_x = 1.5
scale_y = 2
path_counter = 0

a_scale = 10000/130
b_scale = 10000/130

paths.append(('HOME',))

# 0,0 is furthest, then up is less (?)

real_start_mm = (800,800)

orig_length = (real_start_mm[0] * a_scale, real_start_mm[1] * b_scale)
reset(ser)
penup(ser)

for xy_path in tqdm(paths):
    if len(xy_path) == 0:
        continue
    
    print("path %s (%s)" % (path_counter, 100*path_counter/len(paths)))
    path_counter += 1
    
    ab_path = []

    for coord in tqdm(xy_path):
        if coord=='HOME':
            movement = (0,0)
        else:
            coord = (offset_x + coord[0]*scale_x, offset_y + coord[1]*scale_y)
            short_ab_mm = translate_xy_to_ab(coord)
            #print(short_ab_mm)
            short_ab_steps = (short_ab_mm[0] * a_scale, short_ab_mm[1] * b_scale)
            #print(short_ab_steps)
            movement = (int(orig_length[0] - short_ab_steps[0]),int( orig_length[1] - short_ab_steps[1]))
            if movement[0] < 0 or movement[1] < 0:
                print("%s -> %s" % (coord, movement))
                raise Exception("out of bounds")
                
        #print("Going -> %s" % (movement,))
        ab_path.append(movement)
        
        
    #input("> PENUP !\r\n")
    penup(ser)
    set_target('A', ab_path[0][0], ser, output=False)
    set_target('B', ab_path[0][1], ser, output=False)
    gogogo(ser, wait=True, output=False)
        
    pendown(ser)
    counter = 0
    for coord in tqdm(ab_path[1:]):
        counter += 1
        #print("Step %s of %s (%s)" % (counter, len(ab_path), 100*counter/len(ab_path)))
        set_target('A', coord[0], ser, output=False)
        set_target('B', coord[1], ser, output=False)
        gogogo(ser, wait=True, output=False)
    penup(ser)

#print(len(ab_path))
#print(int(offset_x + xy_path[0][0]*scale_x), int(offset_y + xy_path[0][1]*scale_y))

  0%|          | 0/2 [00:00<?, ?it/s]

!-- I received: B
!--- B target length = -16843009
!-- I received: R
path 0 (0.0)
(728.5071787306781, 988.7522138953227) -> (-32934, -16755)


Exception: out of bounds

In [158]:
penup(ser)
set_target('A',1000, ser, output=False)
set_target('B',1000, ser, output=False)
gogogo(ser)

--- Making a move ---
!--- B target length = 1000
!-- I received: G
!--- Running ---


In [118]:
ser.close()

In [155]:
reset(ser)

!-- I received: R


In [216]:
def go_to_xy(target_coord,ser):
    target_lengths = translate_xy_to_ab(target_coord)
    travel_lengths = (reset_point[0] - target_lengths[0], reset_point[1] - target_lengths[1])

    a_step_mm = 10000/125
    b_step_mm = 10000/125

    travel_steps = (int(travel_lengths[0] * a_step_mm), int(travel_lengths[1] * b_step_mm))

    set_target("A", travel_steps[0], ser, output=True)
    set_target("B", travel_steps[1], ser, output=True)
    gogogo(ser, wait=True)

In [219]:
reset(ser)
path = [
    (650, 400),
    (300, 400),
    (300, 150),
    (650, 150),
    (650, 400)
]

for point in path:
    go_to_xy(point,ser)

!-- I received: R
!-- I received: A
!--- A target length = 2942
!-- I received: B
!--- B target length = 23020
--- Making a move ---
!-- I received: G

!--- Running ---

!-- Reached move-end ---

!-- I received: A
!--- A target length = 24000
!-- I received: B
--- Making a move ---
!--- B target length = 1574

!-- I received: G

!--- Running ---

!-- Reached move-end ---

!-- I received: A
!--- A target length = 37167
!-- I received: B
!--- B target length = 9073
--- Making a move ---
!-- I received: G

!--- Running ---

!-- Reached move-end ---

!-- I received: A
!--- A target length = 10633
!-- I received: B
--- Making a move ---
!--- B target length = 35727

!-- I received: G

!--- Running ---

!-- Reached move-end ---

!-- I received: A
!--- A target length = 2942
!-- I received: B
!--- B target length = 23020
--- Making a move ---
!-- I received: G

!--- Running ---

!-- Reached move-end ---



In [217]:
pendown(ser)
